In [10]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref
import ast
import random
from tqdm import tqdm_notebook

df1=pd.read_csv("scores1_freq.csv")
df2=pd.read_csv("scores2_freq.csv")
df1.head() #On vérifie que le fichier contient bien ce qu'il faut

,description,scores,freq
0,US-British actress Sienna Miller poses after u...,"{'US': 0.978723404, 'British': 0.978723404, 'S...","{'France': 0.16666666666666666, 'British': 0.1..."
1,People gather at a shopping mall in the Shatin...,"{'Shatin': 0.8297872340000001, 'Hong Kong': 1....","{'Hong Kong': 0.5, 'Shatin': 0.5}"
2,AFP presents a photo essay of 19 images by pho...,"{'AFP': 1.0, 'Christof Stache': 0.829787234000...","{'AFP': 0.25, 'Slug GERMANY-AGRICULTURE-CATTLE..."
3,A Palestinian worker harvests dates in the Jor...,"{'Palestinian': 0.489361702, 'Jordan Valley': ...","{'West Bank': 0.07142857142857142, ""the West B..."
4,Turkish Trade Minister Ruhsar Pekcan (R) and U...,"{'Turkish': 0.489361702, 'Ruhsar Pekcan': 0.93...","{'Commerce': 0.16666666666666666, 'Ruhsar Pekc..."


In [15]:
##Conversion des strings contenues dans le csv (colonnes score et fréquences) en dictionnaires

liste_scores1=[]
for string in df1.scores:
    liste_scores1.append(ast.literal_eval(string))   

liste_scores2=[]
for string in df2.scores:
    liste_scores2.append(ast.literal_eval(string))    

liste_freq=[]
for string in df1.freq:
    liste_freq.append(ast.literal_eval(string))

In [7]:
#Toute cette cellule est issue de l'autre notebook (création du csv) et permet donc d'avoir toutes les fonctions
#définies dans ce dernier à disposition.


#On importe le fichier contenant les scores de chaque fonction grammaticale
scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}

for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]

#On importe un modèle md pour avoir des mots vectorisés
nlp=spacy.load("en_core_web_md")  
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

def dep_ent(ent, doc):
    """ Retourne la fonction grammaticale :  la 'dep', d'une entité. Cette fonction est nécessaire car elle permet d'affecter
    une dep à une entité composée de plusieurs mots ayant chacun une dep de base.
    Traite aussi le cas particulier des mots étant des conj ou des compound : leur vrai dep et celle du mot auxquels
    ils sont associés en tant que conj ou compound."""
    start= ent.start
    end=ent.end
    for k in range(start,end):
        if doc[k].head.text not in ent.text: 
            if doc[k].dep_=='conj':     
                tok=doc[k]            
                while tok.dep_=='conj':
                    tok=tok.head      
                return(tok.dep_)
            
            if doc[k].dep_=='compound':   
                tok=doc[k]            
                while tok.dep_=='compound':
                    tok=tok.head      
                return(tok.dep_)
            return(doc[k].dep_)    
    return doc[start].dep_

def ent_good_type(ent): #filtre les entités selon leur type
    return (ent.label_ == "PERSON"or ent.label_ == "NORP" or ent.label_ == "ORG" or ent.label_ == "GPE" or ent.label_ == "EVENT" or ent.label_ == "LOC")

def sort_ent(doc):
    """Retourne la liste des entités en les filtrant selon leur type et en les triant de manière à avoir au début de
    la liste les entités ayant des coréférences."""
    ent_coref=[ent for ent in doc.ents if ent._.is_coref and ent_good_type(ent)]
    ent_vanilla=[ent for ent in doc.ents if ent_good_type(ent) and not ent._.is_coref]
    return ent_coref + ent_vanilla

def scores_doc(doc):
    """Retourne le score de chaque entité pour la méthode sans neuralcoref."""
    res={}
    for ent in doc.ents:
        if ent.text not in res.keys():
            res[ent.text]=dict_val[dep_ent(ent,doc)]
        else:
            res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def scores_doc_coref1(doc):
    """Retourne le score de chaque entité pour la méthode de base opérée sur le document resolved.
    Le document resolved est le document de base dans lequel toutes les références à un groupe de mot sont remplacées
    par celui-ci.
    Par exemple : My dad is home. He watches TV devient My dad is home. My dad watches TV.
    En raisonnant avec le nlp sur le document resolved, le nlp va détecter beaucoup plus de fois la même entité.
    Le principal inconvénient est que toutes les références sont remplacées, y compris celles qui ne sont pas associées 
    à des entités mais plutôt à des très longs bouts de phrase qui sont repris par un pronom comme "it"  """
    
    doc=nlp(doc._.coref_resolved)
    res={}
    for ent in doc.ents:
        if ent_good_type(ent):
            if ent.text not in res.keys():
                res[ent.text]=dict_val[dep_ent(ent,doc)]
            else:
                res[ent.text]+=dict_val[dep_ent(ent,doc)]
    return res

def is_in_cluster(ent,cluster):  #détermine si une entité est dans une des mentions d'un cluster
    for span in cluster.mentions:
        if ent.text in span.text:
            return True
    return False



def scores_doc_coref2(doc):
    """Cette méthode utilise neuralcoref mais au lieu d'agir sur le doc resolved, on va chercher a l'interieur des clusters
    associés au document. Pour rappel, un cluster contient un mot et ses références. Ce mot n'est pas forcément une entité.
    Un cluster est de la forme Trump : [Trump, he].
    Pour chaque entité, on va d'abord regarder si elle est coréférencée (donc mentionnée explicitement dans un cluster)
    ou pas.
    Si elle est coréférencée, on va chercher le cluster qui lui est associé.
    Une fois dans ce cluster, on va regarder pour chaque mention (réf à l'entité) si elle contient ou pas des entités.
    Si c'est le cas, on va alors indiquer que cette entité ne doit pas être retraitée par la suite car elle est traitée en
    tant qu'autre mention de l'entité que l'on traite actuellement.
    Par exemple, pour l'entité référencée Trump, on trouve le cluster suivant : Trump : [Trump, Donald Trump, he].
    Alors l'entité Donald Trump va être marquée comme traitée car elle est en fait équivalente à Trump.
    "he" n'étant pas une entité on a pas ce pb pour cette mention la.
    Ensuite on calcule le score de chaque mention (Trump, Donald Trump et he) en se basant sur leur fonction grammaticale.
    Puis on ajoute ces scores à celui de l'entité de base (Trump).
    
    Si l'entité de base n'est pas référencée, c'est un peu le même principe, la seule complexité en plus étant le fait que
    l'entité n'est pas forcément mentionnée telle quelle dans les mentions d'un cluster. 
    Par exemple, si notre entité de base est Sienna Miller, il est possible qu'on ait un cluster de la forme
    The british actress Sienna Miller : [The british actress Sienna Miller, she].
    Dans ce cas il faut rechercher si chaque mention contient l'entité de base avant de procéder comme pour les 
    entités référencées.
    
    """
    clusters=doc._.coref_clusters  #liste des clusters du doc. 
    res={} #le resultat sera un dictionnaire qui permet d'associer un score à chque entité
    ent_treated={} #dictionnaire pour différencier les entités traitées des autres
    ents_sorted=sort_ent(doc)  #on place les entités ayant une coref avant les autres
    for ent in ents_sorted:   
        ent_treated[ent.text]=0  #on initialise en affectant 0 à toutes les entités

    for ent in ents_sorted:     #pour chaque entité
        if ent_treated[ent.text]==0: #si elle n'est pas considérée comme "traitée"
            if ent._.is_coref:  #si elle est coréférencée (donc si elle apparait en tant que mention pour un cluster )
                #print(ent)
                for cluster in clusters:  #on va chercher le cluster associé à l'entité puisqu'elle est coref
                    if ent in cluster.mentions: # on regarde si l'entité est dans les mentions du cluster
                        for span in cluster.mentions: #Mtnt qu'on est dans le bon cluster, on regarde pour chaque span
                            if span.ents != [] : 
                                if ent.has_vector:
                                    max_sim=0
                                    max_span_ent=span.ents[0]
                                    for span_ent in span.ents: #on cherche l'entité dans le span la plus proche de ent
                                        if span_ent.has_vector:
                                            if ent.similarity(span_ent) > max_sim:
                                                max_span_ent=span_ent
                                                max_sim=ent.similarity(span_ent)
                                    ent_treated[max_span_ent.text]=1 #une fois qu'on l'a trouvé on la marque comme traitée
                            else:
                                max_span_ent=span        
                            if ent.text not in res.keys(): 
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)]  #on affecte le score de l'entité
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)] 
            else: #si l'entité n'est pas exactement coréférencée par neuralcoref
                flag=0
                for cluster in clusters:
                    if is_in_cluster(ent,cluster) and ent.label_ != 'NORP': 
                        flag=1
                        for span in cluster.mentions:
                            #print(span)
                            if span.ents != [] :
                                if ent.has_vector:
                                    max_sim=0
                                    max_span_ent=span.ents[0]
                                    for span_ent in span.ents:
                                        if span_ent.has_vector:
                                            if ent.similarity(span_ent) > max_sim:
                                                max_span_ent=span_ent
                                                max_sim=ent.similarity(span_ent)
                                    ent_treated[max_span_ent.text]=1
                            else:
                                max_span_ent=span #on peut peut etre supprimer cette branche else et...
                            if ent.text not in res.keys():
                                res[ent.text]=dict_val[dep_ent(max_span_ent,doc)] #...replacer max_span_ent par span ici
                            else:
                                res[ent.text] += dict_val[dep_ent(max_span_ent,doc)]  #et ici
                if flag==0: #si l'entité n'est vraiment dans aucun cluster   
                    if ent.text not in res.keys():
                        res[ent.text]=dict_val[dep_ent(ent,doc)]
                    else:
                        res[ent.text]+=dict_val[dep_ent(ent,doc)] 
    return res


def freq_dict(L):
    n = len(L)
    L_unique = list(set(L))
    d = {}
    for a in L_unique:
        for b in L:
            if a == b:
                if a in d.keys():
                    d[a] += 1/n
                else:
                    d[a] = 1/n
    return(d)

In [17]:
##Les deux fonctions scores ci-dessous étaient les premières que nous avions écrites.

def score_sim1(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1."""
    res=0
    freq_doc=freq_dict([ent.text for ent in doc.ents])
    score_image=liste_scores1[k]
    freq_image=liste_freq[k]
    for i in score_doc.keys():
        if i in score_image.keys():
            res += (score_doc[i]+score_image[i])/2 - abs(freq_doc[i]-freq_image[i])/2
    return res

def score_sim2(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref2 : on utilise liste_scores2."""
    res=0
    freq_doc=freq_dict([ent.text for ent in doc.ents])
    score_image=liste_scores2[k]
    freq_image=liste_freq[k]
    for i in score_doc.keys():
        if i in score_image.keys():
            res += (score_doc[i]+score_image[i])/2 - abs(freq_doc[i]-freq_image[i])/2
    return res


#Dernière fonction score développée. Elle donne de meilleurs résultats.

def score_sim3(doc,score_doc,k):
    """Prend en paramètre un doc, le fichier de scores associés à ce doc et un entier k correspondant à l'indice de la 
    description avec laquelle on veut comparer le doc. Retourne le score de similarité entre le doc et la description.
    Ici on calcule le score avec la méthode neuralcoref1 : on utilise liste_scores1.
    Par rapport aux fonctions précédentes, ici on a changé le calcul du score de similarité en "normalisant" les scores
    du doc et de l'image, au lieu de les diviser par 2 (ce qui était inutile)"""
    res=0
    score_image=liste_scores1[k]
    max_doc=max(score_doc.values())
    max_im=max(score_image.values())
    for i in score_doc.keys():
        if i in score_image.keys():
            if max_doc==0 or max_im==0:
                return 0
            res += score_doc[i]/max_doc +score_image[i]/max_im
    return res


def related_descr(doc):
    """Retourne la liste des indices correspondant aux descriptions ayant au moins 1 entité en commun avec le doc.
    Permet d'effectuer moins de comparaison dans la fonction best_image."""
    index_list=[]
    for k in range(len(liste_scores1)):
        for ent in doc.ents:
            if ent.text in liste_scores1[k].keys():
                index_list.append(k)
                break
    return index_list

def best_image1(doc):
    """Retourne l'indice de l'image dont la description correspond le plus au doc passé en paramètre.
    La correspondance (matching) se fait avec score_sim1."""
    best_score=0
    best_descr=0
    score_doc=scores_doc_coref1(doc)
    for k in related_descr(doc):
        if score_sim1(doc,score_doc,k) > best_score:
            best_score=score_sim1(doc,score_doc,k)
            best_descr=k
    return best_descr

def best_image2(doc):
    """Retourne l'indice de l'image dont la description correspond le plus au doc passé en paramètre.
    La correspondance (matching) se fait avec score_sim2."""
    score_doc=scores_doc_coref2(doc)
    best_score=0
    best_descr=0
    for k in related_descr(doc):
        if score_sim2(doc,score_doc,k) > best_score:
            best_score=score_sim2(doc,score_doc,k)
            best_descr=k
    return best_descr

def best_image3(doc):
    """Retourne l'indice de l'image dont la description correspond le plus au doc passé en paramètre.
    La correspondance (matching) se fait avec score_sim3."""
    score_doc=scores_doc_coref1(doc)
    best_score=0
    best_descr=0
    for k in related_descr(doc):
        if score_sim3(doc,score_doc,k) > best_score:
            best_score=score_sim3(doc,score_doc,k)
            best_descr=k
    return best_descr

In [21]:
#On crée une liste d'indices associés à 100 descriptions différentes de la base.
rand_ind_list=[]
for i in range(100):
    num = random.randint(0, df1.shape[0])
    while(num in rand_ind_list):
        num = random.randint(0, df1.shape[0])
    rand_ind_list.append(num)
print(rand_ind_list)

#On calcule le pourcentage de descriptions i telles que la description la plus proche de i dans la base soit i
#On a constaté que souvent, lorsqu'une description ne matche pas avec elle-même, elle matche avec une description très
#voisine qui est probablement proche de la description initiale en terme d'indice. En effet, on a souvent par exemple une
#dizaine de photos illustrant le même évènement/fait d'actualité.
#count2 doit remédier en partie à ce pb en étant incrémenté si la description la plus proche de i est i ou est
#proche de i en terme d'indice (entre i-50 et i+50).
count=0
count2=0
rand_int_list=range(100)
for i in tqdm_notebook(rand_ind_list):
    best=best_image3(nlp(df1.description[i]))
    flag=0
    if best==i:
        count+=1
        flag=1
    for k in range(i-50,i+50):
        if best==k:
            count2+=1
            flag=1
            continue
    if flag==0:
        print(i,best) #si aucun compteur n'est incrémenté dans l'itération, potentielle erreur de l'algorithme.
        #On print alors i et best pour voir par la suite si l'erreur était une vraie erreur ou pas.
print(count,'% et ', count2,'%')

[68298, 40293, 3466, 58103, 67638, 35801, 22853, 33636, 26733, 67719, 57352, 77063, 32114, 5974, 70629, 11646, 67801, 50471, 10573, 59194, 81196, 71611, 73512, 1074, 81853, 74660, 54325, 63637, 68800, 2890, 31012, 79269, 52566, 38580, 51315, 54471, 16366, 55348, 47501, 20884, 55982, 33281, 32674, 5708, 28431, 53544, 62748, 61686, 71576, 80723, 69064, 24722, 61944, 19753, 13854, 11466, 15963, 21806, 12000, 41225, 30044, 66182, 4728, 46890, 17706, 57062, 27190, 37619, 77393, 39063, 1625, 11292, 21873, 76304, 11662, 6449, 20544, 4150, 70076, 77159, 47505, 46607, 7857, 63422, 38167, 34983, 34135, 40650, 82191, 34269, 19311, 12463, 78595, 40638, 63480, 65039, 43343, 71759, 23813, 56833]


3466 3945
77063 76625
11646 11723
54325 43869
61686 61884
11466 11355
27190 27127
37619 35968
1625 1515
11292 8862
11662 9592
70076 25900
77159 79316
34983 8701
82191 47126
19311 19247
12463 10770
54 % et  83 %


In [30]:
print(df1.description[1625],'\n',df1.description[1515])
#On voit par exemple ici que malgré plus de 50 indices d'écarts, les descriptions sont quasiment identiques.
#Le pourcentage de réussite de l'algorithme doit alors être encore revu à la hausse.
#Après avoir étudié les descriptions associées aux indices ayant été print à la cellule précédente, on peut en fait
#estimer que le vrai pourcentage de réussite de l'algorithme est supérieur à 90%.
#Les seules erreurs constatées sont liées à  des descriptions identiques pour des dates différentes (on ne prend
#actuellement pas en compte les dates dans les entités) ou bien des descriptions très proches avec seulement la 1ère phrase
#qui diffère. Il faudrait alors accorder plus d'importance à la 1ère phrase.

Colombian President Juan Manuel Santos (L) looks the FIFA World Cup trophy next to France's 1998 World Cup Champion, French-Argentine former footballer David Trezeguet, in Bogota, Colombia, on April 3, 2018 during the FIFA Trophy World Tour ahead of the World Cup to be held in Rusia between June 14 and July 15.   
 Colombian President Juan Manuel Santos (L) holds the FIFA World Cup trophy next to France's 1998 World Cup Champion, French-Argentine former footballer David Trezeguet, in Bogota, Colombia, on April 3, 2018 during the FIFA Trophy World Tour ahead of the World Cup to be held in Rusia between June 14 and July 15.  
